In [1]:
# !pip3 install npy-append-array

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [4]:
import malaya_speech
import malaya_speech.config
import tensorflow as tf
from npy_append_array import NpyAppendArray
import joblib
import numpy as np
import torch
from malaya_speech.train.model import hubert

# tf.compat.v1.enable_eager_execution()






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
config = malaya_speech.config.transducer_featurizer_config
config['feature_type'] = 'mfcc'
config['num_feature_bins'] = 30
config['stride_ms'] = 20
featurizer = malaya_speech.utils.tf_featurization.STTFeaturizer(**config)

In [6]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [7]:
i = tf.placeholder(tf.float32, [None])
v = featurizer.vectorize(i)
v

<tf.Tensor 'truediv_1:0' shape=(?, 30) dtype=float32>

In [8]:
deltas = malaya_speech.utils.tf_featurization.deltas(v)
deltas

<tf.Tensor 'strided_slice_3:0' shape=(?, 30) dtype=float32>

In [9]:
ddeltas = malaya_speech.utils.tf_featurization.deltas(deltas)
ddeltas

<tf.Tensor 'strided_slice_4:0' shape=(?, 30) dtype=float32>

In [10]:
concated = tf.concat([v, deltas, ddeltas], axis = 1)
concated

<tf.Tensor 'concat_1:0' shape=(?, 90) dtype=float32>

In [11]:
sess = tf.Session()

In [12]:
!rm out.npy
feat_f = NpyAppendArray('out.npy')
leng_path = 'out.len'
leng_f = open(leng_path, 'w')

In [13]:
# 281
v1 = sess.run(concated, feed_dict = {i: y})
feat_f.append(v1)
leng_f.write(f"{len(v1)}\n")
v1.shape

(281, 90)

In [14]:
# 175
v1 = sess.run(concated, feed_dict = {i: y1})
feat_f.append(v1)
leng_f.write(f"{len(v1)}\n")
v1.shape

(175, 90)

In [15]:
leng_f.close()

In [16]:
feat = np.load('out.npy', mmap_mode="r")
with open(leng_path, "r") as f:
    lengs = [int(line.rstrip()) for line in f]
    offsets = [0] + np.cumsum(lengs[:-1]).tolist()
feat.shape

(456, 90)

In [17]:
offsets, lengs

([0, 281], [281, 175])

In [18]:
np.random.seed(0)
km_model = hubert.kmeans.get_km_model()
km_model.fit(feat)

Init 1/20 with method: k-means++
Inertia for init 1/20: 188.207199
Init 2/20 with method: k-means++
Inertia for init 2/20: 197.536896
Init 3/20 with method: k-means++
Inertia for init 3/20: 195.492920
Init 4/20 with method: k-means++
Inertia for init 4/20: 200.351501
Init 5/20 with method: k-means++
Inertia for init 5/20: 179.846130
Init 6/20 with method: k-means++
Inertia for init 6/20: 192.046005
Init 7/20 with method: k-means++
Inertia for init 7/20: 183.870071
Init 8/20 with method: k-means++
Inertia for init 8/20: 190.811417
Init 9/20 with method: k-means++
Inertia for init 9/20: 207.396988
Init 10/20 with method: k-means++
Inertia for init 10/20: 185.271866
Init 11/20 with method: k-means++
Inertia for init 11/20: 200.053375
Init 12/20 with method: k-means++
Inertia for init 12/20: 207.223862
Init 13/20 with method: k-means++
Inertia for init 13/20: 187.891174
Init 14/20 with method: k-means++
Inertia for init 14/20: 195.406738
Init 15/20 with method: k-means++
Inertia for init 1

Minibatch iteration 88/100: mean batch inertia: 0.522143, ewa inertia: 0.522143 
Minibatch iteration 89/100: mean batch inertia: 0.518121, ewa inertia: 0.518121 
Minibatch iteration 90/100: mean batch inertia: 0.524661, ewa inertia: 0.524661 
Minibatch iteration 91/100: mean batch inertia: 0.523175, ewa inertia: 0.523175 
Minibatch iteration 92/100: mean batch inertia: 0.519629, ewa inertia: 0.519629 
Minibatch iteration 93/100: mean batch inertia: 0.514391, ewa inertia: 0.514391 
Minibatch iteration 94/100: mean batch inertia: 0.518962, ewa inertia: 0.518962 
Minibatch iteration 95/100: mean batch inertia: 0.519287, ewa inertia: 0.519287 
Minibatch iteration 96/100: mean batch inertia: 0.522211, ewa inertia: 0.522211 
Minibatch iteration 97/100: mean batch inertia: 0.518368, ewa inertia: 0.518368 
Minibatch iteration 98/100: mean batch inertia: 0.525116, ewa inertia: 0.525116 
Minibatch iteration 99/100: mean batch inertia: 0.527789, ewa inertia: 0.527789 
Minibatch iteration 100/100:

MiniBatchKMeans(batch_size=10000, compute_labels=False, max_no_improvement=100,
                n_clusters=100, n_init=20, reassignment_ratio=0.0, verbose=1)

In [19]:
joblib.dump(km_model, 'kmean.km')

['kmean.km']

In [20]:
for offset, leng in zip(offsets, lengs):
    print(feat[offset: offset + leng].shape)

(281, 90)
(175, 90)


In [22]:
kmean = hubert.kmeans.ApplyKmeans_TF('kmean.km')

In [29]:
kmean(feat[offset: offset + leng])

array([98, 98, 24, 24, 70,  2, 66, 66, 14, 66, 14, 14, 14,  0, 85, 47, 47,
       67, 97, 97, 29, 31, 31, 31, 31, 67, 67, 17, 17, 43, 39, 39, 39, 37,
       54, 27, 27, 27, 75, 75, 75, 60, 60, 60, 60,  7,  7,  7, 50, 50, 50,
       50, 50, 50, 50, 50, 50, 50, 33, 33, 33, 33, 26, 81, 81, 81, 46, 99,
       32, 97, 22, 35, 24, 24, 72, 24, 24,  2, 14, 57, 57, 21, 21, 32, 32,
       75, 75, 75, 75, 60, 60, 60, 62,  5, 35, 89,  9, 32, 75, 75, 60, 23,
       23, 61, 61, 61, 61,  5,  5,  5,  5,  5, 71, 71,  2,  2, 45, 45,  2,
       45, 66, 66, 66, 66, 14, 66, 14, 14, 66, 14, 66, 14, 14, 66, 14, 14,
       14, 45, 66, 14, 14, 45, 66, 14, 66, 45, 14, 14, 66, 14, 14, 14, 41,
       45, 14, 66, 66, 66, 66, 14, 45, 66, 66, 66, 66, 66, 66, 66, 66, 14,
       14, 14, 14, 57, 19])

In [24]:
kmean_tf = kmean(concated)
kmean_tf

Instructions for updating:
keep_dims is deprecated, use keepdims instead


<tf.Tensor 'ArgMin:0' shape=(?,) dtype=int64>

In [28]:
sess.run(kmean_tf, feed_dict = {i: y1})

array([98, 98, 24, 24, 70,  2, 66, 66, 14, 66, 14, 14, 14,  0, 85, 47, 47,
       67, 97, 97, 29, 31, 31, 31, 31, 67, 67, 17, 17, 43, 39, 39, 39, 37,
       54, 27, 27, 27, 75, 75, 75, 60, 60, 60, 60,  7,  7,  7, 50, 50, 50,
       50, 50, 50, 50, 50, 50, 50, 33, 33, 33, 33, 26, 81, 81, 81, 46, 99,
       32, 97, 22, 35, 24, 24, 72, 24, 24,  2, 14, 57, 57, 21, 21, 32, 32,
       75, 75, 75, 75, 60, 60, 60, 62,  5, 35, 89,  9, 32, 75, 75, 60, 23,
       23, 61, 61, 61, 61,  5,  5,  5,  5,  5, 71, 71,  2,  2, 45, 45,  2,
       45, 66, 66, 66, 66, 14, 66, 14, 14, 66, 14, 66, 14, 14, 66, 14, 14,
       14, 45, 66, 14, 14, 45, 66, 14, 66, 45, 14, 14, 66, 14, 14, 14, 41,
       45, 14, 66, 66, 66, 66, 14, 45, 66, 66, 66, 66, 66, 66, 66, 66, 14,
       14, 14, 14, 57, 19])